In [ ]:

import os
if not os.path.exists('hw2-dl-audio'):
    !git clone https://github.com/Vladislav747/hw2-dl-audio.git
    %cd hw2-dl-audio
else:
    %cd hw2-dl-audio


In [ ]:

!pip install -q torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0
!pip install -q soundfile numpy==1.26.4 scipy torch_audiomentations tqdm matplotlib pandas
!pip install -q hydra-core wget datasets editdistance pyctcdecode
!pip install -q comet_ml wandb


## 2. Быстрый тест на onebatchtest датасете


In [ ]:
# Запуск обучения на маленьком датасете для проверки
!python3 train.py -cn=deepspeech2 \
  datasets=onebatchtest \
  trainer.n_epochs=1 \
  trainer.override=True \
  writer=cometml


## 3. Обучение на полном датасете (train-clean-360)

Показываю как запускал - это версия без augs

In [ ]:
# Раскомментируйте для запуска полного обучения
# python3 train.py -cn=deepspeech2_360 \
#     datasets=train_clean_360 \
#     trainer.n_epochs=200 \
#     trainer.override=True \
#     writer=cometml \
#     writer.run_name="deepspeech2_360" \
#     2>&1


Версия с augs - сильная вероятность использования

In [ ]:

# python3 train.py -cn=deepspeech2_augs_strong_360 \
#     datasets=train_clean_360 \
#     trainer.n_epochs=200 \
#     trainer.override=True \
#     writer=cometml \
#     writer.run_name="deepspeech2_augs_strong_360" \
#     2>&1

## 4. Запуск Inference


In [ ]:
# Запуск inference на обученной модели
# Убедитесь, что у вас есть сохраненная модель в saved/model_best.pth

#### Оценка модели на test-clean(без augs)
python3 inference.py \
    -cn=inference \
    datasets=test_clean \
    inferencer.from_pretrained="saved/deepspeech2_360/model_best.pth" \
    inferencer.save_path="predictions_test_clean"
 


In [ ]:
#### Оценка модели на test-other(с augs)

python3 inference.py \
    -cn=inference \
    datasets=test_other \
    inferencer.from_pretrained="saved/deepspeech2_augs_strong_360/model_best.pth" \
    inferencer.save_path="predictions_test_other"
